In [1]:
# Python实用宝典
# 2020/05/05
# 转载请注明出处
import datetime
import os.path
import sys
import backtrader as bt
from backtrader.indicators import EMA

# Create a Stratey

class TestStrategy(bt.Strategy):


    
    def __init__(self):
        print("init")

        # 初始化相关数据
        self.dataclose = self.datas[0].close
        
        # 七日移动平均线
        self.sma7 = bt.indicators.SimpleMovingAverage(self.datas[0], period=7)
        #self.sma7 = bt.indicators.SimpleMovingAverage(self.data.close, period=7)
        # 二十一日移动平均线
        self.sma21 = bt.indicators.SimpleMovingAverage(self.datas[0], period=21)
        #self.sma21 = bt.indicators.SimpleMovingAverage(self.data.close, period=21)
        self.buy_signal = bt.indicators.CrossOver(self.sma7,self.sma21)
        
       

    
    def start(self):
        print("start...")
     

    def next(self):
        #print("next")
        #print(self.dataclose[0])
        #正式交易

        # 是否正在下单，如果是的话不能提交第二次订单

        if not self.position and self.buy_signal[0] == 1:
            self.oder = self.buy()
            print("买入价格：" + str(self.dataclose[0]))
    
        if not self.position and self.buy_signal[0] == -1:
            self.order = self.sell()
            print("卖出价格：" + str(self.dataclose[0]))

        if self.position and self.buy_signal[0] == 1:
            self.order = self.close()
            self.order = self.buy()
            print("买入价格：" + str(self.dataclose[0]))
            
        if self.position and self.buy_signal[0] == -1:
            self.order = self.close()
            self.order = self.sell()    
            print("卖出价格：" + str(self.dataclose[0]))
            
if __name__ == '__main__':

    # 初始化模型
    cerebro = bt.Cerebro()

    # 构建策略
    strats = cerebro.addstrategy(TestStrategy)
    # 每次买100股
    #cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    #cerebro.addsizer(percent)
    #modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    #datapath = os.path.join(modpath, "/root/002859.csv")

    # 加载数据到模型中
    data = bt.feeds.YahooFinanceData(dataname='510300.SS',
                                    fromdate=datetime.datetime(2019, 12, 1),
                                    todate=datetime.datetime(2020, 12, 31),)
                                    
    cerebro.adddata(data)
    data2 = cerebro.resampledata(data,timeframe= bt.TimeFrame.Weeks,compression=1)
    #cerebro.adddata(data2)
    # 设定初始资金和佣金
    cerebro.broker.setcash(1000000.0)
    cerebro.broker.setcommission(0.002)
    cerebro.addsizer(bt.sizers.PercentSizer, percents=98)

    # 策略执行前的资金
    print('启动资金: %.2f' % cerebro.broker.getvalue())

    # 策略执行
    cerebro.run()
    print('结束资金: %.2f' % cerebro.broker.getvalue())

    cerebro.plot(style = "candlestick")  # 绘图


启动资金: 1000000.00


SSLError: HTTPSConnectionPool(host='finance.yahoo.com', port=443): Max retries exceeded with url: /quote/510300.SS/history (Caused by SSLError(SSLError("bad handshake: SysCallError(110, 'ETIMEDOUT')")))